In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime
from apscheduler.schedulers.blocking import BlockingScheduler

# List of Indian stock tickers
tickers = ["TATAMOTORS.BO"]

# Function to get stock data and info
def get_stock_data(tickers):
    stock_data = []
    for ticker in tickers:
        try:
            print(f"Fetching data for {ticker}...")
            data = yf.Ticker(ticker)
            
            # Get historical market data for the last day
            history = data.history(period="1d")
            
            # Convert to DataFrame and ensure datetime columns are timezone-naive
            history.index = history.index.tz_localize(None)
            
            # Add a column for the stock ticker
            history['Ticker'] = ticker
            
            # Append the data to the list
            stock_data.append(history)
            
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    
    # Concatenate all the stock data into a single DataFrame
    all_data = pd.concat(stock_data)
    
    return all_data

# Function to save data to CSV and update it
def save_stock_data_to_csv():
    # Get stock data
    all_stock_data = get_stock_data(tickers)
    
    # Load existing data if it exists
    file_exists = os.path.isfile('Tatamotors_stock_data.csv')
    
    if file_exists:
        # Read existing data
        existing_data = pd.read_csv('Tatamotors_stock_data.csv', index_col=0, parse_dates=True)
        # Concatenate new data with existing data
        updated_data = pd.concat([existing_data, all_stock_data])
        # Drop duplicates based on the index
        updated_data = updated_data[~updated_data.index.duplicated(keep='last')]
    else:
        # If the file does not exist, use the new data
        updated_data = all_stock_data
    
    # Save the updated data to CSV
    updated_data.to_csv('Tatamotors_stock_data.csv')
    
    print("Stock data has been updated in 'Tatamotors_stock_data.csv'.")

# Scheduler setup
if __name__ == "__main__":
    scheduler = BlockingScheduler()

    
    # Schedule the stock fetching function to run daily
    scheduler.add_job(save_stock_data_to_csv, 'interval', days=1)
    
    # Start the scheduler
    print("Scheduler started. Fetching stock data daily.")
    save_stock_data_to_csv()  # Fetch data immediately on start
    scheduler.start()


Scheduler started. Fetching stock data daily.
Fetching data for TATAMOTORS.BO...
Stock data has been updated in 'Tatamotors_stock_data.csv'.
